# HMTK Geological Tools Demonstration

This notepad demonstrates the use of the HMTK geological tools for preparing fault source models for input into OpenQuake

## Construction of the Geological Input File

An active fault model input file contains two sections:

1) A tectonic regionalisation - this can provide a container for a set of properties that may be assigned to multiple faults by virtue of a common tectonic region

2) A set of active faults

#### Tectonic Regionalisation Representation in the Fault Source File

In the tectonic regionalisation information each of the three properties can be represented according to a set of weighted values.
For example, in the case below faults in an arbitrarily named tectonic region (called here "GEM Region 1") will share the same set
of magnitude scaling relations and shear moduli, unless over-written by the specific fault. Those faults assigned to "GEM Region 2"
will have the magnitude scaling relation fixed as WC1994 and the shear modulus of 30 GPa

### Active Fault Model

A set of active faults will be defined with a common ID and name. 

An active fault set containing a single fault is shown below:

#### Fault Geometry Representations - Example 1: Simple Fault

#### Fault Geometry Representations - Example 2: Complex Fault

#### Rupture Properties

The rupture requires characterisation of the rake (using the Aki & Richards 2002 convention), the slip-type, the slip completeness factor
(an integer constraining the quality of the slip information with 1 being the hights quality), the range of slip values and their 
corresponding weights, and the aseismic slip coefficient (the proportion of slip released aseismically, 1.0 - coupling coefficient)

## The Magnitude Frequency Distributions

In [1]:
%load_ext autoreload
%autoreload 2
import warnings; warnings.filterwarnings('ignore')

In [2]:
#Import tools
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from openquake.hmtk.plotting.faults.geology_mfd_plot import plot_recurrence_models
from openquake.hazardlib.scalerel.wc1994 import WC1994  
from openquake.hazardlib.scalerel.strasser2010 import StrasserInterface
from openquake.hazardlib.scalerel.strasser2010 import StrasserIntraslab

print ("Everything imported OK!")

Everything imported OK!


The following examples refer to a fault with the following properties:

Length (Along-strike) = 100 km,
Width (Down-Dip) = 20 km,
Slip = 10.0 mm/yr,
Rake = 0. (Strike Slip),
Magnitude Scaling Relation = Wells & Coppersmith (1994),
Shear Modulus = 30.0 GPa

# Set up fault parameters
slip = 10.0  # Slip rate in mm/yr

# Area = along-strike length (km) * down-dip with (km)
area = 100.0 * 20.0

# Rake = 0.
rake = 0.

# Magnitude Scaling Relation
msr = WC1994()

### Anderson & Luco (Arbitrary)

This describes a set of distributons where the maximum magnitude is assumed to rupture the whole fault surface

#Magnitude Frequency Distribution Example

anderson_luco_config1 = {'Model_Name': 'AndersonLucoArbitrary',
                         'Model_Type': 'First',
                         'Model_Weight': 1.0,  # Weight is a required key - normally weights should sum to 1.0 - current example is simply illustrative! 
                         'MFD_spacing': 0.1,
                         'Maximum_Magnitude': None,
                         'Minimum_Magnitude': 4.5,
                         'b_value': [0.8, 0.05]}
anderson_luco_config2 = {'Model_Name': 'AndersonLucoArbitrary',
                         'Model_Type': 'Second',
                         'Model_Weight': 1.0,
                         'MFD_spacing': 0.1,
                         'Maximum_Magnitude': None,
                         'Minimum_Magnitude': 4.5,
                         'b_value': [0.8, 0.05]}
anderson_luco_config3 = {'Model_Name': 'AndersonLucoArbitrary',
                         'Model_Type': 'Third',
                         'Model_Weight': 1.0,   
                         'MFD_spacing': 0.1,
                         'Maximum_Magnitude': None,
                         'Minimum_Magnitude': 4.5,
                         'b_value': [0.8, 0.05]}
# Create a list of the configurations
anderson_luco_arb = [anderson_luco_config1, anderson_luco_config2, anderson_luco_config3]

# View the corresponding magnitude recurrence model
plot_recurrence_models(anderson_luco_arb, area, slip, msr, rake, msr_sigma=0.0)

### Anderson & Luco (Area - MMax)

This describes a set of distributons where the maximum rupture extent is limited to only part of the fault surface

anderson_luco_config1 = {'Model_Name': 'AndersonLucoAreaMmax',
                         'Model_Type': 'First',
                         'Model_Weight': 1.0,  # Weight is a required key - normally weights should sum to 1.0 - current example is simply illustrative! 
                         'MFD_spacing': 0.1,
                         'Maximum_Magnitude': None,
                         'Minimum_Magnitude': 4.5,
                         'b_value': [0.8, 0.05]}
anderson_luco_config2 = {'Model_Name': 'AndersonLucoAreaMmax',
                         'Model_Type': 'Second',
                         'Model_Weight': 1.0,
                         'MFD_spacing': 0.1,
                         'Maximum_Magnitude': None,
                         'Minimum_Magnitude': 4.5,
                         'b_value': [0.8, 0.05]}
anderson_luco_config3 = {'Model_Name': 'AndersonLucoAreaMmax',
                         'Model_Type': 'Third',
                         'Model_Weight': 1.0,   
                         'MFD_spacing': 0.1,
                         'Maximum_Magnitude': None,
                         'Minimum_Magnitude': 4.5,
                         'b_value': [0.8, 0.05]}

# For these models a displacement to length ratio is needed
disp_length_ratio = 1.25E-5

# Create a list of the configurations
anderson_luco_area_mmax = [anderson_luco_config1, anderson_luco_config2, anderson_luco_config3]

# View the corresponding magnitude recurrence model
plot_recurrence_models(anderson_luco_area_mmax, area, slip, msr, rake, msr_sigma=0.0)


### Characteristic Earthquake

The following example illustrates a "Characteristic" Model, represented by a Truncated Gaussian Distribution

characteristic = [{'Model_Name': 'Characteristic',
                   'MFD_spacing': 0.05,
                   'Model_Weight': 1.0,
                   'Maximum_Magnitude': None,
                   'Sigma': 0.15,  # Standard Deviation of Distribution (in Magnitude Units) - omit for fixed value
                   'Lower_Bound': -3.0,   # Bounds of the distribution correspond to the number of sigma for truncation
                   'Upper_Bound': 3.0}]

# View the corresponding magnitude recurrence model
plot_recurrence_models(characteristic, area, slip, msr, rake, msr_sigma=0.0)

### Youngs & Coppersmith (1985) Models

The following describes the recurrence from two distributions presented by Youngs & Coppersmith (1985): 1) Exponential Distribution, 2) Hybrid Exponential-Characteristic Distribution

exponential = {'Model_Name': 'YoungsCoppersmithExponential',
               'MFD_spacing': 0.1,
               'Maximum_Magnitude': None,
               'Maximum_Magnitude_Uncertainty': None,
               'Minimum_Magnitude': 5.0,
               'Model_Weight': 1.0,
               'b_value': [0.8, 0.1]}

hybrid = {'Model_Name': 'YoungsCoppersmithCharacteristic',
          'MFD_spacing': 0.1,
          'Maximum_Magnitude': None,
          'Maximum_Magnitude_Uncertainty': None,
          'Minimum_Magnitude': 5.0,
          'Model_Weight': 1.0,
          'b_value': [0.8, 0.1],
          'delta_m': None}

youngs_coppersmith = [exponential, hybrid]

# View the corresponding magnitude recurrence model
plot_recurrence_models(youngs_coppersmith, area, slip, msr, rake, msr_sigma=0.0)


## Epistemic Uncertainty Examples

This example considers the fault defined at the top of the page. This fault defines two values of slip rate and two different magnitude frequency distributions

In [3]:
def show_file_contents(filename):
    """
    Shows the file contents
    """
    fid = open(filename, 'r')
    for row in fid.readlines():
        print(row)
    fid.close()

input_file = 'input_data/Simple_Faults_ASC.toml'
#input_file = 'input_data/Simple_Faults_Interface.toml'
#input_file = 'input_data/Simple_Faults_Intraslab.toml'
show_file_contents(input_file)

 Fault_Model_ID = 1

Fault_Model_Name = "Faults"



[[Fault_Model]]

ID = 1

Tectonic_Region = "Active Shallow Crust"

Fault_Name = "West Valley Fault"

Rake = 0

Slip_Type = "Strikeslip"

Slip_Completeness_Factor = 1

Aseismic = 0.02

Aspect_Ratio = 1.5



  [Fault_Model.Fault_Geometry]

  Fault_Typology = "Simple"

  Fault_Trace = [

  121.053168,

  14.17022148,

  121.0413762,

  14.19176247,

  121.046193,

  14.21734547,

  121.0400986,

  14.25356815,

  121.0404498,

  14.32828173,

  121.0474568,

  14.42331193,

  121.0574187,

  14.52583089,

  121.0802395,

  14.63251614,

  121.1383568,

  14.76058873,

  121.1472366,

  14.79273591,

  121.1604828,

  14.8589442,

  121.1824943,

  14.91851788,

  121.1944125,

  14.96467329,

  121.201101,

  14.99834189

]

  Upper_Depth = 0

  Lower_Depth = 30

  Dip = 70



  [Fault_Model.Slip]

  Value = [ 3, 6, 10, 12, 20 ]

  Weight = [ 0.15, 0.25, 0.25, 0.25, 0.1 ]



  [[Fault_Model.MFD_Model]]

  Model_Name = "YoungsCoppersmithC

  13.15232524,

  121.1351541,

  13.12808463,

  121.1467089,

  13.09692961,

  121.1502489,

  13.09087199,

  121.1511608,

  13.0830825,

  121.1582997,

  13.05711917,

  121.1715676,

  13.03635146,

  121.1821565,

  13.02597009,

  121.2001273,

  12.99549727,

  121.2017594,

  12.97676072,

  121.2079537,

  12.96236911,

  121.2097905,

  12.95159516,

  121.2143599,

  12.93287371,

  121.2139212,

  12.92885486,

  121.2211158,

  12.91311707,

  121.230932,

  12.89820559,

  121.2371386,

  12.88452317,

  121.24674,

  12.87146166,

  121.2556946,

  12.82036129,

  121.2763454,

  12.78070029,

  121.2896715,

  12.75245201,

  121.3010652,

  12.73405479,

  121.3155121,

  12.72803795,

  121.3243346,

  12.71146406,

  121.3442079,

  12.6849767,

  121.3815149,

  12.69057589,

  121.3940441,

  12.67292644,

  121.4019239,

  12.65979728,

  121.4183934,

  12.62245122,

  121.4272767,

  12.59125246,

  121.4479784,

  12.54198881,

  121.4564412,

  12.51221604


  123.4958439,

  10.7144156,

  123.4961875,

  10.71380618,

  123.4965285,

  10.71325398,

  123.4967713,

  10.71243288,

  123.4970474,

  10.71165655,

  123.4972906,

  10.71101329,

  123.4977422,

  10.70809666,

  123.4976922,

  10.70764511,

  123.4976254,

  10.70684504,

  123.4974252,

  10.70604994,

  123.4973374,

  10.70551231,

  123.4972273,

  10.70477612,

  123.4972067,

  10.70402962,

  123.4969922,

  10.70347752,

  123.496395,

  10.7010737,

  123.4963271,

  10.70052384,

  123.4962302,

  10.6997225,

  123.496163,

  10.69860881,

  123.4961628,

  10.69809522,

  123.4961656,

  10.69748968,

  123.4961283,

  10.69684408,

  123.4961386,

  10.69615353,

  123.4960864,

  10.69537258,

  123.4959868,

  10.69489696,

  123.4960559,

  10.69491487,

  123.4959459,

  10.69447187,

  123.4958644,

  10.69311769,

  123.4958364,

  10.69226677,

  123.4958129,

  10.69138678,

  123.4957704,

  10.69054498,

  123.495687,

  10.69019925,

  123.4955347

### Example 1 - Full Enumeration

In this example each individual MFD for each branch is determined. In the resulting file the fault is duplicated n_branches number of times, with the
corresponding MFD multiplied by the end-branch weight

In [4]:
# Import the Parser
from openquake.hmtk.parsers.faults.fault_yaml_parser import FaultYmltoSource

# Fault mesh discretization step
mesh_spacing = 2.5 #(km)

# Read in the fault model
reader = FaultYmltoSource(input_file)
fault_model, tectonic_region = reader.read_file(mesh_spacing)

# Construct the fault source model (this is really running the MFD calculation code)
fault_model.build_fault_model()

# Write to an output NRML file
output_file_1 = 'output_data/Simple_Faults_ASC_Full.xml'
#output_file_1 = 'output_data/Simple_Faults_Interface_Full.xml'
#output_file_1 = 'output_data/Simple_Faults_Intraslab_Full.xml'

fault_model.source_model.serialise_to_nrml(output_file_1)

show_file_contents(output_file_1)

<?xml version="1.0" encoding="utf-8"?>

<nrml

xmlns="http://openquake.org/xmlns/nrml/0.5"

xmlns:gml="http://www.opengis.net/gml"

>

    <sourceModel

    name="Faults"

    >

        <sourceGroup

        rup_interdep="indep"

        src_interdep="indep"

        tectonicRegion="Active Shallow Crust"

        >

            <simpleFaultSource

            id="008_1"

            name="PFZ Burias Segment"

            >

                <simpleFaultGeometry>

                    <gml:LineString>

                        <gml:posList>

                            123.4004765 12.70776811 123.2667777 12.87624766 123.220207 12.91035886 123.1555019 12.96302817 123.1096187 13.0354893 123.0698584 13.11704146 122.9673016 13.23295505 122.8608598 13.37308986

                        </gml:posList>

                    </gml:LineString>

                    <dip>

                        90

                    </dip>

                    <upperSeismoDepth>

                        0

       

                    <lowerSeismoDepth>

                        30

                    </lowerSeismoDepth>

                </simpleFaultGeometry>

                <magScaleRel>

                    WC1994

                </magScaleRel>

                <ruptAspectRatio>

                    1.5

                </ruptAspectRatio>

                <incrementalMFD

                binWidth="0.1"

                minMag="4.8"

                >

                    <occurRates>

                        0.09698152667901695 0.0715795586859125 0.05283102253718725 0.03899321249760793 0.028779882498268315 0.02124168755433885 0.015677940665091438 0.011571482862147655 0.008540612475152311 0.006303605364990275 0.004652528224777635 0.0034339108540285433 0.0025344808636767768 0.0018706348304900885 0.0013806672282253107 0.0010190348025306119 0.0007521232542785784 0.0005551227379298506 0.00040972174761743925 0.000302405034059155 0.0002231973410152137 0.00016473618963140866 0.00012158752452353664 8

                </simpleFaultGeometry>

                <magScaleRel>

                    WC1994

                </magScaleRel>

                <ruptAspectRatio>

                    1.5

                </ruptAspectRatio>

                <incrementalMFD

                binWidth="0.1"

                minMag="4.8"

                >

                    <occurRates>

                        0.0013515887496327443 0.0011771845895243828 0.0010252849161331331 0.0008929858312831315 0.0007777581453942743 0.000677399026430179 0.0005899898878923837 0.0005138597107965354 0.0004475530984491456 0.00038980245332123354 0.00033950374411835033 0.00029569539978084873 0.0002575399269266268 0.00022430789931304155 0.00019536401324119388 0.001318658371407446

                    </occurRates>

                </incrementalMFD>

                <rake>

                    0.0

                </rake>

            </simpleFaultSource>

            <simpleFaultSource

            id="2_2"

            n


            </simpleFaultSource>

            <simpleFaultSource

            id="6_10"

            name="Lubang Fault South"

            >

                <simpleFaultGeometry>

                    <gml:LineString>

                        <gml:posList>

                            120.3179809 13.57234523 120.5157907 13.58108539 121.0480144 13.52499716 121.2856311 13.47745492 121.4926154 13.39525712

                        </gml:posList>

                    </gml:LineString>

                    <dip>

                        90

                    </dip>

                    <upperSeismoDepth>

                        0

                    </upperSeismoDepth>

                    <lowerSeismoDepth>

                        30

                    </lowerSeismoDepth>

                </simpleFaultGeometry>

                <magScaleRel>

                    WC1994

                </magScaleRel>

                <ruptAspectRatio>

                    1.5

                </rup

### Example 2: Collapsed Branches

In the following example we implement the same model, this time collapsing the branched. This means that the MFD is discretised and the incremental rate
in each magnitude bin is the weighted sum of the rates in that bin from all the end branches of the logic tree.

When collapsing the branches, however, it is necessary to define a single Magnitude Scaling Relation that will need to be assigned to the fault for
use in OpenQuake.

In [5]:
# Read in the fault model
reader = FaultYmltoSource(input_file)
fault_model, tectonic_region = reader.read_file(mesh_spacing)

# Scaling relation for export
output_msr = WC1994()
#output_msr = StrasserInterface()
#output_msr = StrasserIntraslab()

# Construct the fault source model - collapsing the branches
fault_model.build_fault_model(collapse=True, rendered_msr=output_msr)


# Write to an output NRML file
output_file_2 = 'output_data/Simple_Faults_ASC_Collapsed.xml'
#output_file_2 = 'output_data/Simple_Faults_Interface_Collapsed.xml'
#output_file_2 = 'output_data/Simple_Faults_Intraslab_Collapsed.xml'

fault_model.source_model.serialise_to_nrml(output_file_2)

show_file_contents(output_file_2)

<?xml version="1.0" encoding="utf-8"?>

<nrml

xmlns="http://openquake.org/xmlns/nrml/0.5"

xmlns:gml="http://www.opengis.net/gml"

>

    <sourceModel

    name="Faults"

    >

        <sourceGroup

        rup_interdep="indep"

        src_interdep="indep"

        tectonicRegion="Active Shallow Crust"

        >

            <simpleFaultSource

            id="008_1"

            name="PFZ Burias Segment"

            >

                <simpleFaultGeometry>

                    <gml:LineString>

                        <gml:posList>

                            123.4004765 12.70776811 123.2667777 12.87624766 123.220207 12.91035886 123.1555019 12.96302817 123.1096187 13.0354893 123.0698584 13.11704146 122.9673016 13.23295505 122.8608598 13.37308986

                        </gml:posList>

                    </gml:LineString>

                    <dip>

                        90

                    </dip>

                    <upperSeismoDepth>

                        0

       

### Example 3: Gulf of Corinth Faults

In the last example we will take some real fault data from the Gulf of Corinth

In [ ]:
# For plotting
from openquake.hmtk.plotting.mapping import HMTKBaseMap
corinth_fault_file = "input_data/GulfOfCorinth_Faults.yml"

reader = FaultYmltoSource(corinth_fault_file)
corinth_faults, tectonic_region = reader.read_file(mesh_spacing)

print "           ID                         Name   Area (km^2)"
for i, fault in enumerate(corinth_faults.faults):
    print "%2g    %s    %25s    %10.3f" % (i, fault.id, fault.name, fault.area)

For convenience we will collapse the branches again

In [ ]:
# Scaling relation for export
output_msr = WC1994()

# Construct the fault source model - collapsing the branches
corinth_faults.build_fault_model(collapse=True, rendered_msr=output_msr)

Let's take a look at the fault sources

In [ ]:
map_config = {'min_lon': 20.9, 'max_lon': 24.0,
              'min_lat': 37.5, 'max_lat': 39.0, 'resolution':'h'}

# Map the Source
src_basemap = HMTKBaseMap(map_config, "Gulf of Corinth Faults")
src_basemap.add_source_model(corinth_faults.source_model)

##### Find and Show Events within 30 km of the Xylocastro Fault

In [ ]:
from openquake.hmtk.parsers.catalogue.csv_catalogue_parser import CsvCatalogueParser
from openquake.hmtk.seismicity.selector import CatalogueSelector

# Load in the catalogue
catalogue_file = "input_data/Aegean_ExtendedCat1.csv"
parser = CsvCatalogueParser(catalogue_file)
catalogue = parser.read_file()
print catalogue.get_number_events()

#Create a selector object
selector = CatalogueSelector(catalogue)
xylocastro_fault = corinth_faults.faults[15]

# Select earthquakes within 30 km (rupture distance) of the fault
xylocastro_fault.select_catalogue(selector, 30.0, distance_metric="rupture")
print "%g events with 30 km of the Xylocastro Fault" % xylocastro_fault.catalogue.get_number_events()

Show the events

In [ ]:
map_config = {'min_lon': 20.9, 'max_lon': 24.0,
              'min_lat': 37.5, 'max_lat': 39.0, 'resolution':'h'}

# Map the Source
src_basemap = HMTKBaseMap(map_config, "Gulf of Corinth Faults")
src_basemap.add_source_model(corinth_faults.source_model, overlay=True)
src_basemap.add_catalogue(xylocastro_fault.catalogue)

### Export output to XML File

In [ ]:
corinth_output_file = "output_data/New_Gulf_Of_Corinth_Faults.xml"
corinth_faults.source_model.serialise_to_nrml(corinth_output_file)
show_file_contents(corinth_output_file)